In [ ]:
import OCC.Core.BRepBndLib

import OCC.Core.BRep
import OCC.Core.BRepPrimAPI
import OCC.Core.BRepAlgoAPI
import OCC.Core.BRepBuilderAPI

import OCC.Core.GProp
import OCC.Core.BRepGProp

import ifcopenshell
import ifcopenshell.geom

# Specify to return pythonOCC shapes from ifcopenshell.geom.create_shape()
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

# Initialize the viewer
from ifc_viewer import ifc_viewer        
viewer = ifc_viewer()


# Open the IFC file using IfcOpenShell
ifc_file = ifcopenshell.open("models/IfcOpenHouse.ifc")

# Display the geometrical contents of the file using Python OpenCascade
products = ifc_file.by_type("IfcProduct")
for product in products:
    if product.is_a("IfcOpeningElement"): continue
    if product.Representation:
        shape = ifcopenshell.geom.create_shape(settings, product)
        viewer.Add(product, shape.geometry, shape.styles)

viewer.Display()

In [ ]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)
settings.set(settings.DISABLE_OPENING_SUBTRACTIONS, True)

viewer2 = ifc_viewer()

# Get a list of all walls in the file
walls = ifc_file.by_type("IfcWall")

# Create a list of wall representation shapes
# and compute the bounding box of these shapes
wall_shapes = []
bbox = OCC.Core.Bnd.Bnd_Box()
for wall in walls:
    shape = ifcopenshell.geom.create_shape(settings, wall)
    
    wall_shapes.append((wall, shape))
    OCC.Core.BRepBndLib.brepbndlib_Add(shape.geometry, bbox)    
    
    viewer2.Add(wall, shape.geometry, shape.styles)

# Calculate the center/average of the bounding box
bounding_box_center = ifcopenshell.geom.utils.get_bounding_box_center(bbox)
print("Bounding box center: %.2f %.2f %.2f" % (
    bounding_box_center.X(), 
    bounding_box_center.Y(),
    bounding_box_center.Z())
)

viewer2.DisplayShape([bounding_box_center])

viewer2.Display()

In [ ]:
import OCC.Core.TopExp
import OCC.Core.TopTools
import OCC.Core.BRepTools
import OCC.Core.TopoDS

from OCC.Core.TopAbs import *

def yield_subshapes(s, type=None, vertices=False):
    """
    Based on type in `s` and additional keyword arguments
    initialize an OCCT iterator or explorer and yield its
    values.
    """
    if isinstance(s, OCC.Core.TopTools.TopTools_IndexedMapOfShape):
        for i in range(1, s.Size() + 1):
            yield s.FindKey(i)
        return
    elif isinstance(s, OCC.Core.TopTools.TopTools_ListOfShape):
        it = OCC.Core.TopTools.TopTools_ListIteratorOfListOfShape(s)
    else:
        if type is None:
            if s.ShapeType() == TopAbs_WIRE:
                it = OCC.Core.BRepTools.BRepTools_WireExplorer(s)
            else:
                it = OCC.Core.TopoDS.TopoDS_Iterator(s)
        else:
            it = OCC.Core.TopExp.TopExp_Explorer(s, type)
    while it.More():
        if isinstance(it, OCC.Core.BRepTools.BRepTools_WireExplorer) and vertices:
            yield it.CurrentVertex()
        elif hasattr(it, 'Value'):
            yield it.Value()
        else:
            yield it.Current()
        it.Next()

In [ ]:
viewer3 = ifc_viewer()

# Now create halfspace solids from the inner faces of the wall
halfspaces = []
for wall, shape in wall_shapes:
    for face in yield_subshapes(shape.geometry, TopAbs_FACE):
        surf = OCC.Core.BRep.BRep_Tool.Surface(face)
        assert surf.DynamicType().Name() == "Geom_Plane"

        P, V = OCC.Core.gp.gp_Pnt(), OCC.Core.gp.gp_Vec()
        prop = OCC.Core.BRepGProp.BRepGProp_Face(face)
        prop.Normal(0, 0, P, V)
                    
        if abs(V.Z()) < 1.e-5:
            face_bbox = OCC.Core.Bnd.Bnd_Box()
            OCC.Core.BRepBndLib.brepbndlib_Add(face, face_bbox)
            face_center = ifcopenshell.geom.utils.get_bounding_box_center(face_bbox).XYZ()
            
            face_towards_center = bounding_box_center.XYZ() - face_center
            face_towards_center.Normalize()
            
            dot = face_towards_center.Dot(V.XYZ())
            
            if dot < -0.8:
                
                viewer3.DisplayShape(face)
                new_face = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeFace(surf, 1.e-5).Face()
                halfspace = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeHalfSpace(new_face, bounding_box_center).Solid()
                halfspaces.append(halfspace)

viewer3.Display()

In [ ]:
viewer4 = ifc_viewer()

# Create halfspace solids from the bottom faces of the roofs
roofs = ifc_file.by_type("IfcRoof")
for roof in roofs:
    for roof_slab in roof.IsDecomposedBy[0].RelatedObjects:
        shape = ifcopenshell.geom.create_shape(settings, roof_slab).geometry
        
        for face in yield_subshapes(shape, TopAbs_FACE):
            surf = OCC.Core.BRep.BRep_Tool.Surface(face)
            P, V = OCC.Core.gp.gp_Pnt(), OCC.Core.gp.gp_Vec()
            prop = OCC.Core.BRepGProp.BRepGProp_Face(face)
            prop.Normal(0, 0, P, V)
            if V.Z() < -0.7:
                new_face = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeFace(surf, 1.e-5).Face()
                halfspace = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeHalfSpace(new_face, bounding_box_center).Solid()
                halfspaces.append(halfspace)
            
# Create an initial box from which to cut the halfspaces
common_shape = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeBox(OCC.Core.gp.gp_Pnt(-10, -10, 0), OCC.Core.gp.gp_Pnt(10, 10, 10)).Solid()
for halfspace in halfspaces:
    common_shape = OCC.Core.BRepAlgoAPI.BRepAlgoAPI_Common(common_shape, halfspace).Shape()    

viewer4.DisplayShape(common_shape)

# Calculate the volume properties of the resulting space shape
props = OCC.Core.GProp.GProp_GProps()
OCC.Core.BRepGProp.brepgprop_VolumeProperties(common_shape, props)
print("Space volume: %.3f cubic meter" % props.Mass())

viewer4.Display()